In [112]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

import re
import requests
!pip install folium
import folium
from bs4 import BeautifulSoup as BS

Defaulting to user installation because normal site-packages is not writeable


In [128]:
# 식당 데이터 임포트

name_data1 = pd.read_csv("서울관광재단_식당품질정보_20230111 - 금천구.csv", encoding='cp949')
name_data1.rename(columns = {'Unnamed: 0':'식당명',
                            'Unnamed: 1':'지역명',
                            'Unnamed: 2':'네이버평점'}, inplace=True)
name_data1.dropna(axis=0, inplace=True)
name_data1.head()
 

,식당명,지역명,네이버평점
0,오늘 한판,금천구,5.0
1,중한홍원면점,금천구,5.0
2,솔다방,금천구,5.0
3,팔팔식당,금천구,5.0
4,우리식당,금천구,5.0


In [104]:
name_data2 = pd.read_csv("서울관광재단_식당품질정보_20230111 - 동대문구.csv", encoding='cp949')
name_data2.rename(columns = {'Unnamed: 0':'식당명',
                            'Unnamed: 1':'지역명',
                            'Unnamed: 2':'네이버평점'}, inplace=True)
name_data2.dropna(axis=0, inplace=True)
name_data2.head()

,식당명,지역명,네이버평점
0,제일식당,동대문구,5.0
1,바니양념통닭,동대문구,5.0
2,목포식당,동대문구,5.0
3,동원다방,동대문구,5.0
4,빈대떡왕대포,동대문구,5.0


In [105]:
name_data3 = pd.read_csv("서울관광재단_식당품질정보_20230111 - 마포구.csv", encoding='cp949')
name_data3.rename(columns = {'Unnamed: 0':'식당명',
                            'Unnamed: 1':'지역명',
                            'Unnamed: 2':'네이버평점'}, inplace=True)
name_data3.dropna(axis=0, inplace=True)
name_data3.head()

,식당명,지역명,네이버평점
0,젠바,마포구,5.0
1,서울가든호텔 카페1883,마포구,5.0
2,마당호프,마포구,5.0
3,묵을래,마포구,5.0
4,로뎀식당,마포구,5.0


In [106]:
name_data4 = pd.read_csv("서울관광재단_식당품질정보_20230111 - 서초구.csv", encoding='cp949')
name_data4.rename(columns = {'Unnamed: 0':'식당명',
                            'Unnamed: 1':'지역명',
                            'Unnamed: 2':'네이버평점'}, inplace=True)
name_data4.dropna(axis=0, inplace=True)
name_data4.head()

,식당명,지역명,네이버평점
0,작은마을,서초구,5.0
1,우도,서초구,5.0
2,즉석강남떡볶이,서초구,5.0
3,쿡앤펍 플러스 보드게임 콘솔게임,서초구,5.0
4,03수제비칼국수,서초구,5.0


In [107]:
name_data5 = pd.read_csv("서울관광재단_식당품질정보_20230111 - 성동구.csv", encoding='cp949')
name_data5.rename(columns = {'Unnamed: 0':'식당명',
                            'Unnamed: 1':'지역명',
                            'Unnamed: 2':'네이버평점'}, inplace=True)
name_data5.dropna(axis=0, inplace=True)
name_data5.head()

,식당명,지역명,네이버평점
0,풀코스,성동구,5.0
1,충남식당,성동구,5.0
2,실비식당,성동구,5.0
3,무등산닭한마리,성동구,5.0
4,소점식당,성동구,5.0


In [137]:
#지역명이 포함된 식당명을 변수로 지정
items1 = name_data1['식당명']
print(items1[:1000],"\n")

items2 = name_data2['식당명']
print(items2[:1000],"\n")

items3 = name_data3['식당명']
print(items3[:1000],"\n")

items4 = name_data4['식당명']
print(items4[:1000],"\n")

items5 = name_data5['식당명']
print(items5[:1000],"\n")

0         오늘 한판
1        중한홍원면점
2           솔다방
3          팔팔식당
4          우리식당
         ...   
995       오징어나라
996        돼지분식
997     프랑세즈과자점
998    포장마차200호
999     풍년쌈밥보리밥
Name: 식당명, Length: 1000, dtype: object 

0        제일식당
1      바니양념통닭
2        목포식당
3        동원다방
4      빈대떡왕대포
        ...  
995     에이셉피자
996     동마곱창집
997     카페이너프
998      수빈마마
999     원조보신탕
Name: 식당명, Length: 1000, dtype: object 

0                 젠바
1      서울가든호텔 카페1883
2               마당호프
3                묵을래
4               로뎀식당
           ...      
995              형제집
996              비에니
997              씨스루
998         후니지니 마카롱
999               도나
Name: 식당명, Length: 1000, dtype: object 

0                   작은마을
1                     우도
2                즉석강남떡볶이
3      쿡앤펍 플러스 보드게임 콘솔게임
4               03수제비칼국수
             ...        
995                울엄마분식
996                  섬마을
997                 커피정원
998                 공간21
999                 two평
Name: 식당명, Length: 1000, dtype: object 



In [ ]:
searching = items1[:1000]
len(searching)
for i in searching:
    n=0
    n=n+1
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching[n])
    headers = {
        "Authorization": "KakaoAK baccb3812bb7865995e93c318b31e10e"
    }
    time.sleep(2)
    places_p = requests.get(url, headers = headers).json()['documents'][0]
    places[i] = places_p

places

In [135]:
searching = items1[:1000]

url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching[6])
headers = {
    "Authorization": "KakaoAK baccb3812bb7865995e93c318b31e10e"
}
time.sleep(2)
places_p = requests.get(url, headers = headers).json()['documents'][0]


places_p

{'address_name': '서울 용산구 한강로2가 135-1',
 'category_group_code': 'FD6',
 'category_group_name': '음식점',
 'category_name': '음식점 > 술집 > 호프,요리주점',
 'distance': '',
 'id': '8806992',
 'phone': '02-794-9163',
 'place_name': '그린호프',
 'place_url': 'http://place.map.kakao.com/8806992',
 'road_address_name': '서울 용산구 한강대로 126',
 'x': '126.9696151016053',
 'y': '37.530638117305664'}

In [146]:
searching = items1[:1000]
for i in searching:
    n=0
    
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching[n])
    headers = {
        "Authorization": "KakaoAK baccb3812bb7865995e93c318b31e10e"
    }
    time.sleep(2)
    places_p = requests.get(url, headers = headers).json()['documents'][0]
    places[i] = places_p
    
    n=n+1

places

1000